In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from ndlinear import NdLinear
import torch.nn.functional as F

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


train_set = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform
)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)


test_set = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform
)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:

class NdLinearCNN(nn.Module):
    def __init__(self):
        super(NdLinearCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.ndlinear1 = NdLinear(input_dims=(64*8*8,), hidden_size=(256,))
        self.ndlinear2 = NdLinear(input_dims=(256,), hidden_size=(10,))

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.ndlinear1(x)
        x = F.relu(x)
        x = self.ndlinear2(x)
        return x
    
    
model = NdLinearCIFAR10()


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:

for epoch in range(50):
    model.train()
    total, correct = 0, 0

for inputs, labels in train_loader:
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = model(inputs)  
    loss = criterion(outputs, labels)  
    loss.backward()  
    optimizer.step()  

    preds = outputs.argmax(dim=1)  
    correct += (preds == labels).sum().item()
    total += labels.size(0)

    acc = correct / total
    print(f"Epoch {epoch + 1}: Accuracy = {acc:.2%}")

Epoch 50: Accuracy = 9.38%
Epoch 50: Accuracy = 14.06%
Epoch 50: Accuracy = 14.58%
Epoch 50: Accuracy = 16.02%
Epoch 50: Accuracy = 15.94%
Epoch 50: Accuracy = 15.89%
Epoch 50: Accuracy = 16.52%
Epoch 50: Accuracy = 17.38%
Epoch 50: Accuracy = 17.71%
Epoch 50: Accuracy = 17.97%
Epoch 50: Accuracy = 18.18%
Epoch 50: Accuracy = 18.36%
Epoch 50: Accuracy = 18.99%
Epoch 50: Accuracy = 18.97%
Epoch 50: Accuracy = 19.48%
Epoch 50: Accuracy = 19.82%
Epoch 50: Accuracy = 19.85%
Epoch 50: Accuracy = 19.53%
Epoch 50: Accuracy = 20.07%
Epoch 50: Accuracy = 20.31%
Epoch 50: Accuracy = 20.31%
Epoch 50: Accuracy = 20.81%
Epoch 50: Accuracy = 21.33%
Epoch 50: Accuracy = 21.81%
Epoch 50: Accuracy = 22.00%
Epoch 50: Accuracy = 22.42%
Epoch 50: Accuracy = 22.34%
Epoch 50: Accuracy = 22.27%
Epoch 50: Accuracy = 22.47%
Epoch 50: Accuracy = 22.29%
Epoch 50: Accuracy = 22.38%
Epoch 50: Accuracy = 22.36%
Epoch 50: Accuracy = 22.40%
Epoch 50: Accuracy = 22.38%
Epoch 50: Accuracy = 22.50%
Epoch 50: Accuracy = 

In [ ]:
torch.save(model.state_dict(), "ndlinear_cifar_model.pth")

NameError: name 'model' is not defined